In [28]:
import requests
import json
import pprint
import pandas as pd
import csv
import re
from datetime import datetime

- Get all shops

In [2]:
all_shops_API = requests.get("https://www.foody.vn/__get/Place/HomeListPlace?page=5&lat=10.823099&lon=106.629664&count=400&districtId=&cateId=&cuisineId=&type=1", 
                            headers={'X-Requested-With': 'XMLHttpRequest',
                                            'Cookie': 'floc=217'})
data = all_shops_API.text
dataJson = json.loads(data)
print(len(dataJson['Items']))

0


In [39]:
def extractshop(page, count, lat, long):
    while count > 0:
        url = f'https://www.foody.vn/__get/Place/HomeListPlace?page={page}&lat={lat}&lon={long}&count={count}&districtId=&cateId=&cuisineId=&type=1'
        all_shops_API = requests.get(url, headers={'X-Requested-With': 'XMLHttpRequest',
                                                'Cookie': 'floc=217'})
        data = all_shops_API.text
        dataJson = json.loads(data)
        if len(dataJson['Items']) != 0:
            break
        else: 
            count = count - 1    
           
    
    shop_review = []
    CityId = dataJson['CityId']
    
    for shop in dataJson['Items']: 
        converted_date = shop['ResCreatedOn']
        converted_date = converted_date[7:-2]
        converted_date = int(converted_date)
        converted_date = datetime.fromtimestamp(converted_date // 1000000000)
        converted_date = converted_date.strftime('%d/%m/%Y')
        shop_review.append({'CityId': CityId, 
                            'name': shop['Name'],
                            'addr': shop['Address'],
                            'RestaurantId': shop['RestaurantId'],
                            'RestaurantStatus': shop['RestaurantStatus'],
                            'avgrating' : shop['AvgRating'],
                            'TotalPictures' : shop['TotalPictures'],
                            'TotalSaves' : shop['TotalSaves'],
                            'TotalReviews' : shop['TotalReviews'],
                            'TotalFavourites' : shop['TotalFavourites'],
                            'TotalViews' : shop['TotalViews'],
                            'TotalCheckIns' : shop['TotalCheckIns'],
                            'CreatedOn'  : converted_date,
                            'Latitude'  : shop['Latitude'],
                            'Longitude'  : shop['Longitude'],
                            'ResUrlReviews' : shop['ResUrlReviews']})
    return shop_review

lmao = extractshop(5, 500, 10.823099, 106.629664)
lmao

[{'CityId': 217,
  'name': 'Cô Ngọc - Bún Thang ',
  'addr': '443/63 Điện Biên Phủ, P. 3, Quận 3, TP. HCM',
  'RestaurantId': 99908,
  'RestaurantStatus': 2,
  'avgrating': 5.6,
  'TotalPictures': 14,
  'TotalSaves': 36,
  'TotalReviews': 5,
  'TotalFavourites': 0,
  'TotalViews': 0,
  'TotalCheckIns': 0,
  'CreatedOn': '02/01/1970',
  'Latitude': 10.771735,
  'Longitude': 106.679242,
  'ResUrlReviews': '/ho-chi-minh/co-ngoc-bun-thang/binh-luan'},
 {'CityId': 217,
  'name': 'Quán chay Hoa Tâm',
  'addr': '1 Đường Số 1 , P. 7, Quận Gò Vấp, TP. HCM',
  'RestaurantId': 29848,
  'RestaurantStatus': 2,
  'avgrating': 8.0,
  'TotalPictures': 9,
  'TotalSaves': 15,
  'TotalReviews': 2,
  'TotalFavourites': 0,
  'TotalViews': 0,
  'TotalCheckIns': 0,
  'CreatedOn': '02/01/1970',
  'Latitude': 10.819912,
  'Longitude': 106.686237,
  'ResUrlReviews': '/ho-chi-minh/quan-chay-hoa-tam/binh-luan'},
 {'CityId': 217,
  'name': 'Kobe Teppanyaki - Nhà Hàng Nhật',
  'addr': '13A Tú Xương, P. 7, Quận 3, T

In [40]:
df = pd.DataFrame.from_dict(lmao) 
df.to_csv (r'allshop.csv', index = False, header=True, encoding='utf-8')

- Get menu of ONE shop: fải loop qua từng request_id trong cell code trên

In [5]:
# VD: có thể thay request_id = 1031704 để thử
request_id = 1031704
delivery_dish_API = requests.get(f"https://gappapi.deliverynow.vn/api/dish/get_delivery_dishes?request_id={request_id}&id_type=1", 
                                    headers={'x-foody-api-version': '1',
                                            'x-foody-app-type': '1004',
                                            'x-foody-client-language': 'vi',
                                            'x-foody-client-type': '1',
                                            'x-foody-client-version': '1',
                                            'X-Foody-Client-Id': 'cookies.__ondemand_sessionid'
                                        }
                                )
data = delivery_dish_API.text
dataJson = json.loads(data)
print(dataJson)

{'reply': {'menu_infos': [{'dish_type_id': 2776277, 'dish_type_name': 'SIÊU SALE', 'dishes': [{'description': '', 'price': {'text': '22,000đ', 'unit': 'đ', 'value': 22000.0}, 'is_active': True, 'photos': [{'width': 120, 'value': 'https://images.foody.vn/res/g104/1031704/s120x120/94e72ee0-a2eb-450d-ae6a-50ffde95-306f05ee-220106142045.jpeg', 'height': 120}, {'width': 180, 'value': 'https://images.foody.vn/res/g104/1031704/s180x180/94e72ee0-a2eb-450d-ae6a-50ffde95-306f05ee-220106142045.jpeg', 'height': 180}, {'width': 400, 'value': 'https://images.foody.vn/res/g104/1031704/s400x400/94e72ee0-a2eb-450d-ae6a-50ffde95-306f05ee-220106142045.jpeg', 'height': 400}, {'width': 560, 'value': 'https://images.foody.vn/res/g104/1031704/s570x570/94e72ee0-a2eb-450d-ae6a-50ffde95-306f05ee-220106142045.jpeg', 'height': 560}, {'width': 750, 'value': 'https://images.foody.vn/res/g104/1031704/s750x750/94e72ee0-a2eb-450d-ae6a-50ffde95-306f05ee-220106142045.jpeg', 'height': 750}, {'width': 1242, 'value': 'http

- get detail of ONE shop

In [40]:
delivery_dish_API = requests.get("https://gappapi.deliverynow.vn/api/delivery/get_detail?request_id={request_id}&id_type=1", 
                                    headers={'x-foody-api-version': '1',
                                            'x-foody-app-type': '1004',
                                            'x-foody-client-language': 'vi',
                                            'x-foody-client-type': '1',
                                            'x-foody-client-version': '1',
                                            'X-Foody-Client-Id': 'cookies.__ondemand_sessionid'
                                        }
                                )
data = delivery_dish_API.text
dataJson = json.loads(data)
print(dataJson)

{'reply': {'delivery_detail': {'total_order': 0, 'rating': {'total_review': 500, 'avg': 4.7, 'display_total_review': '500+', 'app_link': 'foody://foody.vn?action=reviewlist&id=1031704'}, 'is_subscribe': False, 'is_favorite': False, 'city_id': 217, 'phones': ['0776969606'], 'restaurant_id': 1031704, 'district_id': 16, 'brand_id': 11607, 'video': None, 'asap_is_available': False, 'contract_type': 1, 'id': 121227, 'location_url': 'ho-chi-minh', 'foody_service_id': 1, 'is_quality_merchant': True, 'available_times': [{'date': '2022-09-21', 'times': ['09:55', '10:00', '10:05', '10:10', '10:15', '10:20', '10:25', '10:30', '10:35', '10:40', '10:45', '10:50', '10:55', '11:00', '11:05', '11:10', '11:15', '11:20', '11:25', '11:30', '11:35', '11:40', '11:45', '11:50', '11:55', '12:00', '12:05', '12:10', '12:15', '12:20', '12:25', '12:30', '12:35', '12:40', '12:45', '12:50', '12:55', '13:00', '13:05', '13:10', '13:15', '13:20', '13:25', '13:30', '13:35', '13:40', '13:45', '13:50', '13:55', '14:00',